<a href="https://colab.research.google.com/github/Jaeek27/Data-Science-Projects/blob/main/Association_Rules.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules 

In [ ]:
data = pd.read_excel('/content/Online Retail Python.xlsx')

In [ ]:
data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
data.columns

Index(['InvoiceNo', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'UnitPrice', 'CustomerID', 'Country'],
      dtype='object')

In [ ]:
data.shape

(541909, 8)

In [ ]:
# Checking any missing values 
data.isnull().values.any()

True

In [ ]:
data.isnull().sum()

InvoiceNo           0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
UnitPrice           0
CustomerID     135080
Country             0
dtype: int64

In [ ]:
# Data Pre Processing 
# Stripping extra spaces in the description 
data['Description'] = data['Description'].str.strip() 
# Dropping the rows without any invoice number 
data.dropna(axis = 0, subset =['InvoiceNo'], inplace = True) 
data['InvoiceNo'] = data['InvoiceNo'].astype('str') 
  
# Dropping all transactions which were done on credit 
data = data[~data['InvoiceNo'].str.contains('C')] 

In [ ]:
# Let's see the countries in our dataset

data.Country.unique() 

In [ ]:
# Splitting the data according to the region of transaction
# Transactions done in France 
basket_France = (data[data['Country'] =="France"] 
          .groupby(['InvoiceNo', 'Description'])['Quantity'] 
          .sum().unstack().reset_index().fillna(0) 
          .set_index('InvoiceNo')) 

In [ ]:
# Defining the hot encoding function to make the data suitable  

def hot_encode(x): 
    if(x<= 0): 
        return 0
    if(x>= 1): 
        return 1

Suppose we want to analyze the market trend to France!

In [ ]:
# Applying one hot encoding 

basket_encoded = basket_France.applymap(hot_encode) 
basket_France = basket_encoded 

In [ ]:
basket_France.head()

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Building the model 
frq_items = apriori(basket_France, min_support = 0.1, use_colnames = True) 
  
# Collecting the inferred rules in a dataframe 
rules = association_rules(frq_items, metric ="lift", min_threshold = 1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False]) 

In [ ]:
print(rules.head()) 

                                 antecedents                      consequents  \
40           (SET/6 RED SPOTTY PAPER PLATES)    (SET/6 RED SPOTTY PAPER CUPS)   
43  (SET/6 RED SPOTTY PAPER PLATES, POSTAGE)    (SET/6 RED SPOTTY PAPER CUPS)   
34       (STRAWBERRY LUNCH BOX WITH CUTLERY)                        (POSTAGE)   
26      (ROUND SNACK BOXES SET OF4 WOODLAND)                        (POSTAGE)   
41             (SET/6 RED SPOTTY PAPER CUPS)  (SET/6 RED SPOTTY PAPER PLATES)   

    antecedent support  consequent support   support  confidence      lift  \
40            0.127551            0.137755  0.122449    0.960000  6.968889   
43            0.107143            0.137755  0.102041    0.952381  6.913580   
34            0.122449            0.765306  0.114796    0.937500  1.225000   
26            0.158163            0.765306  0.147959    0.935484  1.222366   
41            0.137755            0.127551  0.122449    0.888889  6.968889   

    leverage  conviction  
40  0.104878   21

From the above output, it can be seen that paper cups and plates are bought together in France. This is because the French have a culture of having a get-together with their friends and family atleast once a week